In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing the Libraries 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px 
from matplotlib import pyplot
from pandas.plotting import autocorrelation_plot
from pandas import DataFrame

import warnings
warnings.filterwarnings("ignore")

# Reading the Data

In [ ]:
series = pd.read_csv("/kaggle/input/covid19-confirmed-cases-kerala/Confirmed Covid 19 Cases Kerala.csv")

# Viewing the Data

In [ ]:
series

# Dimensions of the Data

In [ ]:
series.shape

In [ ]:
series.head()

In [ ]:
series.tail()

In [ ]:
series.sample(10)

In [ ]:
series.set_index('Date',inplace=True) 

In [ ]:
series.describe()

In [ ]:
series.describe().T

In [ ]:
series.info()

In [ ]:
series.plot()
pyplot.show()

In [ ]:
from pandas.plotting import autocorrelation_plot
autocorrelation_plot(series['Confirmed'])
plt.show()

In [ ]:
from statsmodels.tsa.stattools import adfuller

test_result=adfuller(series['Confirmed'])

#Ho: It is non stationary
#H1: It is stationary

def adfuller_test(Confirmed):
    result=adfuller(Confirmed)
    labels = ['ADF Test Statistic','p-value','#Lags Used','Number of Observations Used']
    for value,label in zip(result,labels):
        print(label+' : '+str(value) )
    if result[1] <= 0.05:
        print("strong evidence against the null hypothesis(Ho), reject the null hypothesis. Data has no unit root and is stationary")
    else:
        print("weak evidence against null hypothesis, time series has a unit root, indicating it is non-stationary ")

adfuller_test(series['Confirmed'])


In [ ]:

#from statsmodels.tsa.stattools import adfuller
#from numpy import log
#result = adfuller(series.value.dropna())
#print('ADF Statistic: %f' % result[0])
#print('p-value: %f' % result[1])


from statsmodels.tsa.stattools import adfuller
test_result=adfuller(series['Confirmed'])
test_result
print('ADF Statistic: %f' % test_result[0])
print('p-value: %f' % test_result[1])



In [ ]:
from statsmodels.tsa.arima_model import ARIMA

# 1,1,2 ARIMA Model
model = ARIMA(series['Confirmed'], order=(1,1,2)) 
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
model = ARIMA(series['Confirmed'], order=(1,1,1))
model_fit = model.fit(disp=0)
print(model_fit.summary())

In [ ]:
# Plot residual errors
residuals = pd.DataFrame(model_fit.resid)
fig, ax = plt.subplots(1,2)
residuals.plot(title="Residuals", ax=ax[0])
residuals.plot(kind='kde', title='Density', ax=ax[1])
plt.show()


In [ ]:
# Actual vs Fitted
model_fit.plot_predict(dynamic=False)
plt.show()

In [ ]:
from statsmodels.graphics.tsaplots import plot_acf,plot_pacf
import statsmodels.api as sm

fig = plt.figure(figsize=(12,8))
ax1 = fig.add_subplot(211)
fig = sm.graphics.tsa.plot_acf(series['Confirmed'].iloc[:],lags=40,ax=ax1) 

In [ ]:
model_fit.plot_predict(dynamic=False)


# After April the Covid Cases in Kerala are Increased. 

# How to do Auto Arima Forecast in Python

In [ ]:
pip install pmdarima

In [ ]:
from statsmodels.tsa.arima_model import ARIMA
import pmdarima as pm

model = pm.auto_arima(series['Confirmed'], start_p=1, start_q=1,
                      test='adf',       # use adftest to find optimal 'd'
                      max_p=3, max_q=3, # maximum p and q
                      m=1,              # frequency of series
                      d=None,           # let model determine 'd'
                      seasonal=False,   # No Seasonality
                      start_P=0, 
                      D=0, 
                      trace=True,
                      error_action='ignore',  
                      suppress_warnings=True, 
                      stepwise=True)

print(model.summary())


- Now we have best p,d,q values

In [ ]:
model.plot_diagnostics(figsize=(7,5))
plt.show()

In [ ]:
# Forecast

n_periods = 24
fc, confint = model.predict(n_periods=n_periods, return_conf_int=True)
index_of_fc = np.arange(len(series['Confirmed']), len(series['Confirmed'])+n_periods)

# make series for plotting purpose
fc_series = pd.Series(fc, index=index_of_fc)
lower_series = pd.Series(confint[:, 0], index=index_of_fc)
upper_series = pd.Series(confint[:, 1], index=index_of_fc)

# Plot
plt.plot(series['Confirmed'])
plt.plot(fc_series, color='darkgreen')
plt.fill_between(lower_series.index, 
                 lower_series, 
                 upper_series, 
                 color='k', alpha=.15)

plt.title("Final Forecast")
plt.show()

 # How to automatically build SARIMA model in python

In [ ]:
# Import
data = series['Confirmed']

# Plot
fig, axes = plt.subplots(2, 1, figsize=(10,5), dpi=100, sharex=True)

# Usual Differencing
axes[0].plot(data[:], label='Original Series')
axes[0].plot(data[:].diff(1), label='Usual Differencing')
axes[0].set_title('Usual Differencing')
axes[0].legend(loc='upper left', fontsize=10)


# Seasinal Dei
axes[1].plot(data[:], label='Original Series')
axes[1].plot(data[:].diff(12), label='Seasonal Differencing', color='green')
axes[1].set_title('Seasonal Differencing')
plt.legend(loc='upper left', fontsize=10)

plt.show()

Let’s build the SARIMA model using pmdarima‘s auto_arima(). To do that, you need to set seasonal=True, set the frequency m=31 for month day wise series and enforce D=1.



In [ ]:
!pip install pmdarima

import pmdarima as pm

# Seasonal - fit stepwise auto-ARIMA

smodel = pm.auto_arima(data, start_p=1, start_q=1,
                         test='adf',
                         max_p=3, max_q=3, m=31,
                         start_P=0, seasonal=True,
                         d=None, D=1, trace=True,
                         error_action='ignore',  
                         suppress_warnings=True, 
                         stepwise=True)

smodel.summary()

Let’s forecast for the next 24 months.

# Get Back to that Notebook Soon.

# Show your Support to the Notebook.